In [1]:
import mysql
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [2]:
def create_msql_database(name_database):
    """Function create MYSQL database to save into db config file"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    else:
        print("Database exists")
    mycursor.close()
    
def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(file_dir, table_name):
    """execute query to insert tsv file into table
    :file_dir:: file path
    :table_name:: name of table to be insterted
    :skip_header:: use header 
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('creating MYSQL temporary table...')
        temp_table = f"""CREATE TEMPORARY TABLE temp_table LIKE {table_name}"""
        cursor.execute(temp_table)

        print('creating MYSQL table...')
        load_data_query = f"""LOAD DATA INFILE '{file_dir}' INTO TABLE temp_table
                                FIELDS TERMINATED BY '\t'
                                LINES TERMINATED BY '\n'
                                IGNORE 1 ROWS
                                (@net_id, @net_name, @cidr, @net_range, @net_type, @asn, @org_cust_id, @updated)
                                SET
                                    net_id = NULLIF(@net_id, ' '),
                                    net_name = NULLIF(@net_name, ' '),
                                    cidr = NULLIF(@cidr, ' '),
                                    net_range = NULLIF(@net_range, ' '),
                                    net_type = NULLIF(@net_type, ' '),
                                    asn = NULLIF(@asn, ' '),
                                    org_cust_id = NULLIF(@org_cust_id, ' '),
                                    updated = NULLIF(@updated, ' ');
                                    """
       
        cursor.execute(load_data_query)

        conn.commit()
        print('MYSQL table created')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()



In [3]:
"""                               ON DUPLICATE KEY UPDATE
                                    net_name = VALUES(net_name),
                                    cidr = VALUES(cidr),
                                    net_range = VALUES(net_range),
                                    net_type = VALUES(net_type),
                                    asn = VALUES(asn),
                                    org_cust_id = VALUES(org_cust_id),
                                    updated = VALUES(updated)"""

'                               ON DUPLICATE KEY UPDATE\n                                    net_name = VALUES(net_name),\n                                    cidr = VALUES(cidr),\n                                    net_range = VALUES(net_range),\n                                    net_type = VALUES(net_type),\n                                    asn = VALUES(asn),\n                                    org_cust_id = VALUES(org_cust_id),\n                                    updated = VALUES(updated)'

In [6]:

# Create the table (if it doesn't exist)
network_table = """CREATE TABLE network (net_id VARCHAR(255) UNIQUE, \
                                            net_name VARCHAR(255), cidr VARCHAR(255) UNIQUE, \
                                                net_range VARCHAR(255), net_type VARCHAR(255), \
                                                    asn VARCHAR(255), org_cust_id VARCHAR(255);"""
organization_table = """CREATE TABLE organization (org_id VARCHAR(255) UNIQUE, org_name VARCHAR(255);"""
customer_table = """CREATE TABLE customer (cust_id VARCHAR(255) UNIQUE, cust_name VARCHAR(255);"""

In [8]:
########
#create_msql_database('ThinkCX')
create_msql_table(network_table)
#create_msql_table(organization_table)
#create_msql_table(customer_table)

Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 1


In [8]:
insert_msql_table('/Users/hople/working_folder/Bootcamp_practices/SQL_practice/arin_data_sets/arin_net.tsv', 'network')

Connecting to MYSQL database...
Connected to MYSQL database
creating MYSQL temporary table...
creating MYSQL table...
Error:  1062 (23000): Duplicate entry 'net-108-63-0-0-1' for key 'temp_table.net_id'


In [ ]:
"""#drop dulicate and update new values when there is duplicate in net_id and cidr
tempt_df = df_net[df_net.duplicated(subset=['net_id'], keep=False)]

for _, row in tempt_df.iterrows():
    duplicate_mask = df_net['net_id'] == row['net_id']
    df_net.loc[duplicate_mask, ['net_name', 'cidr', 'net_range', 'net_type', 'asn', 'org_cust_id']] = row[['net_name', 'cidr', 'net_range', 'net_type', 'asn', 'org_cust_id']]

    #drop dulicate and update new values when there is duplicate in net_id and cidr
tempt_df = drop_netid_dup_df[drop_netid_dup_df.duplicated(subset=['cidr'], keep=False)]

for _, row in tempt_df.iterrows():
    duplicate_mask = drop_netid_dup_df['cidr'] == row['cidr']
    drop_cidr_dup_df = drop_netid_dup_df.copy()
    drop_cidr_dup_df.loc[duplicate_mask, ['net_id', 'net_name', 'net_range', 'net_type', 'asn', 'org_cust_id']] = row[['net_id', 'net_name', 'net_range', 'net_type', 'asn', 'org_cust_id']]

    
    """

In [1]:
import pandas as pd
df_net = pd.read_csv('./arin_data_sets/arin_net.tsv', sep='\t')
df_org = pd.read_csv('./arin_data_sets/arin_org.tsv', sep='\t', encoding='latin1')
df_cust = pd.read_csv('./arin_data_sets/arin_cust.tsv', sep='\t', encoding='latin1')

#drop dulicate and update new values when there is duplicate in net_id and cidr
df_net = df_net.drop_duplicates(subset=['net_id'], keep='last')
df_net = df_net.drop_duplicates(subset=['cidr'], keep='last')

#drop duplicate org_id
df_org = df_org.drop_duplicates(subset=['org_id'], keep='last')
#drop duplicate cust_id
df_cust = df_cust.drop_duplicates(subset=['cust_id'], keep='last')

In [2]:
df_net.shape , df_org.shape , df_cust.shape

((65114, 7), (6321, 2), (43238, 2))

In [3]:
df_net

,net_id,net_name,cidr,net_range,net_type,asn,org_cust_id
0,net-104-129-224-0-1,aece-net,104.129.224.0/20,104.129.224.0 - 104.129.239.255,direct assignment,as00000,ai-335
1,net-104-129-96-0-1,xplornet-015,104.129.96.0/19,104.129.96.0 - 104.129.127.255,direct allocation,as22995,barre-2
2,net-104-157-108-0-1,telus-fibre-clgrab22,104.157.108.0/22,104.157.108.0 - 104.157.111.255,reassigned,as852,c07972290
3,net-104-157-79-0-1,telus-hsia-ftmmab3,104.157.79.0/26,104.157.79.0 - 104.157.79.63,reassigned,as852,c06849631
4,net-104-171-48-0-1,cikcable,104.171.48.0/20,104.171.48.0 - 104.171.63.255,direct allocation,"as54614, as174, as6949",cikte
...,...,...,...,...,...,...,...
66515,net-99-254-120-0-1,hsi,99.254.120.0/23,99.254.120.0 - 99.254.121.255,reassigned,NaN,c02172893
66516,net-99-255-10-0-1,hsi,99.255.10.0/23,99.255.10.0 - 99.255.11.255,reassigned,NaN,c02171770
66517,net-99-255-228-0-1,hsi,99.255.228.0/23,99.255.228.0 - 99.255.229.255,reassigned,NaN,c02174354
66518,net-99-255-230-0-1,hsi,99.255.230.0/23,99.255.230.0 - 99.255.231.255,reassigned,NaN,c02174355


In [4]:
df_org

,org_id,org_name
0,807cit,807-city
1,aad-2,aad
2,ablcan-1,abl canada inc.
3,adcogn,adcognito inc.
4,aedint,aed internet inc.
...,...,...
6320,xplor-2,xplornet limited
6321,xtc-5,xbase technologies corp.
6322,youngm,young monkey
6323,zenla-7,zenlayer inc


In [5]:
df_cust

,cust_id,cust_name
0,c00001342,canada ports corporation
1,c00001343,canada ports corporation
2,c00001888,stentor canadian network management
3,c00001991,weldwood of canada limited
4,c00002827,mcmillan bathurst
...,...,...
43233,c08030323,telus-fibre-schlbc01
43234,c08030382,telus-fibre-clgrab09
43235,c08030568,telus-dsl-srrybc01
43236,c08030664,telus-fibre-klwnbc02


In [6]:
#Number of CIDRs for the following org_cust_id: "bdio"

filtered_net = df_net[df_net['org_cust_id'] == "bdio"]
filtered_net.shape

(4286, 7)

In [7]:
#Number of CIDRs where the following value is in the asn field: "as852"

filtered_asn = df_net[df_net['asn'].str.contains("as852", na=False)]
filtered_asn.shape

(2413, 7)

In [8]:
#Provide list of top 10 organization names ranked in descending order of number of CIDRs
# (ties should be broken by ascending alphabetic order)

net_org_df = pd.merge(df_net, df_org, how='left', left_on='org_cust_id', right_on='org_id')

In [9]:
org_cidr_counts = net_org_df.groupby('org_name')['cidr'].count()
org_cidr_counts.sort_values(ascending=False).head(10)

org_name
bell dsl internet ontario                       4286
bell dsl internet quebec                        1517
sasktel wide area network engineering center     975
ovh hosting, inc.                                792
distributel communications ltd.                  371
shared services canada                           336
bell mobility, inc.                              219
bell dsl internet - atlantic                     176
ovh (nwk)                                        166
rogers communications canada inc.                125
Name: cidr, dtype: int64

In [10]:
#Count total number of IPs for the following organization name "bell dsl internet ontario"
#Hint: The following link describes how to convert CIDR to IP count. https://ramprasadtech.com/networking-basics-bits-subnets-network-masks/

# Function to convert CIDR to IP count
def cidr_to_ip_count(cidr):
    return 2**(32-int(cidr.split('/')[1])) - 2


net_org_df['ip_count'] = net_org_df['cidr'].apply(cidr_to_ip_count)
net_org_df[net_org_df['org_name'] == 'bell dsl internet ontario']['ip_count'].sum()

1088644

In [11]:
net_org_df

,net_id,net_name,cidr,net_range,net_type,asn,org_cust_id,org_id,org_name,ip_count
0,net-104-129-224-0-1,aece-net,104.129.224.0/20,104.129.224.0 - 104.129.239.255,direct assignment,as00000,ai-335,ai-335,aequitas innovations inc.,4094
1,net-104-129-96-0-1,xplornet-015,104.129.96.0/19,104.129.96.0 - 104.129.127.255,direct allocation,as22995,barre-2,barre-2,xplornet communications inc.,8190
2,net-104-157-108-0-1,telus-fibre-clgrab22,104.157.108.0/22,104.157.108.0 - 104.157.111.255,reassigned,as852,c07972290,NaN,NaN,1022
3,net-104-157-79-0-1,telus-hsia-ftmmab3,104.157.79.0/26,104.157.79.0 - 104.157.79.63,reassigned,as852,c06849631,NaN,NaN,62
4,net-104-171-48-0-1,cikcable,104.171.48.0/20,104.171.48.0 - 104.171.63.255,direct allocation,"as54614, as174, as6949",cikte,cikte,cik telecom inc,4094
...,...,...,...,...,...,...,...,...,...,...
65109,net-99-254-120-0-1,hsi,99.254.120.0/23,99.254.120.0 - 99.254.121.255,reassigned,NaN,c02172893,NaN,NaN,510
65110,net-99-255-10-0-1,hsi,99.255.10.0/23,99.255.10.0 - 99.255.11.255,reassigned,NaN,c02171770,NaN,NaN,510
65111,net-99-255-228-0-1,hsi,99.255.228.0/23,99.255.228.0 - 99.255.229.255,reassigned,NaN,c02174354,NaN,NaN,510
65112,net-99-255-230-0-1,hsi,99.255.230.0/23,99.255.230.0 - 99.255.231.255,reassigned,NaN,c02174355,NaN,NaN,510


In [12]:
#Provide list of organization names with greater than 100 registered CIDRs. The list should be ranked in descending order of average IP count per CIDR.
#(ties should be broken by ascending alphabetic order)

# calculate cidr count
#net_org_df = net_org_df.copy()
net_org_df['cidr_count'] = net_org_df.groupby('org_name')['cidr'].transform('count')
# Calculate total Ip count for each org
net_org_df['ip_count_per_org'] = net_org_df.groupby('org_name')['ip_count'].transform('sum')
# Calculate the average IP count per CIDR
net_org_df['avg_ip_per_cidr'] = net_org_df['ip_count_per_org'] / net_org_df['cidr_count']


In [13]:
net_org_df

,net_id,net_name,cidr,net_range,net_type,asn,org_cust_id,org_id,org_name,ip_count,cidr_count,ip_count_per_org,avg_ip_per_cidr
0,net-104-129-224-0-1,aece-net,104.129.224.0/20,104.129.224.0 - 104.129.239.255,direct assignment,as00000,ai-335,ai-335,aequitas innovations inc.,4094,1.0,4094.0,4094.000000
1,net-104-129-96-0-1,xplornet-015,104.129.96.0/19,104.129.96.0 - 104.129.127.255,direct allocation,as22995,barre-2,barre-2,xplornet communications inc.,8190,84.0,485336.0,5777.809524
2,net-104-157-108-0-1,telus-fibre-clgrab22,104.157.108.0/22,104.157.108.0 - 104.157.111.255,reassigned,as852,c07972290,NaN,NaN,1022,NaN,NaN,NaN
3,net-104-157-79-0-1,telus-hsia-ftmmab3,104.157.79.0/26,104.157.79.0 - 104.157.79.63,reassigned,as852,c06849631,NaN,NaN,62,NaN,NaN,NaN
4,net-104-171-48-0-1,cikcable,104.171.48.0/20,104.171.48.0 - 104.171.63.255,direct allocation,"as54614, as174, as6949",cikte,cikte,cik telecom inc,4094,25.0,156366.0,6254.640000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65109,net-99-254-120-0-1,hsi,99.254.120.0/23,99.254.120.0 - 99.254.121.255,reassigned,NaN,c02172893,NaN,NaN,510,NaN,NaN,NaN
65110,net-99-255-10-0-1,hsi,99.255.10.0/23,99.255.10.0 - 99.255.11.255,reassigned,NaN,c02171770,NaN,NaN,510,NaN,NaN,NaN
65111,net-99-255-228-0-1,hsi,99.255.228.0/23,99.255.228.0 - 99.255.229.255,reassigned,NaN,c02174354,NaN,NaN,510,NaN,NaN,NaN
65112,net-99-255-230-0-1,hsi,99.255.230.0/23,99.255.230.0 - 99.255.231.255,reassigned,NaN,c02174355,NaN,NaN,510,NaN,NaN,NaN


In [15]:
#filter org with cidr over 100
filtered_net_org_df = net_org_df[net_org_df['cidr_count'] > 100]

#Sort the filtered net org df
sorted_filtered = filtered_net_org_df.sort_values(by='avg_ip_per_cidr', ascending=False)

In [18]:
sorted_filtered.drop_duplicates(subset=['org_name'])

,net_id,net_name,cidr,net_range,net_type,asn,org_cust_id,org_id,org_name,ip_count,cidr_count,ip_count_per_org,avg_ip_per_cidr
47515,net-70-24-0-0-1,bellcanada-18,70.24.0.0/13,70.24.0.0 - 70.31.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,8829976.0,76120.482759
35138,net-206-191-0-0-1,rdc-206-191-0,206.191.24.0/22,206.191.24.0 - 206.191.27.255,direct allocation,as812,rcc-193,rcc-193,rogers communications canada inc.,1022,125.0,5867014.0,46936.112000
6955,net-192-197-175-0-1,bcteleac8,192.197.175.0/24,192.197.175.0 - 192.197.175.255,direct allocation,NaN,tace,tace,telus communications inc.,254,107.0,4979754.0,46539.757009
13236,net-198-103-232-0-1,gtis-c1-232,198.103.232.0/24,198.103.232.0 - 198.103.232.255,reassigned,NaN,ssc-299,ssc-299,shared services canada,254,336.0,4510048.0,13422.761905
8389,net-158-69-135-0-1,ovh-cust-1554491,158.69.135.0/29,158.69.135.0 - 158.69.135.7,reassigned,as16276,ho-2,ho-2,"ovh hosting, inc.",6,792.0,976352.0,1232.767677
36452,net-66-49-136-0-1,dtel-windsor-tec4-v4-03,66.49.136.0/23,66.49.136.0 - 66.49.137.255,reassigned,as11814,distri-47,distri-47,distributel communications ltd.,510,371.0,445138.0,1199.832884
57787,net-69-158-211-0-1,bel1120080513-ca,69.158.211.0/24,69.158.211.0 - 69.158.211.255,reassigned,as577,bellm-8,bellm-8,"bell mobility, inc.",254,219.0,256410.0,1170.821918
479,net-142-167-92-0-1,dsl-142-167-92-0-1600798480,142.167.92.0/22,142.167.92.0 - 142.167.95.255,reassigned,as855,bdia,bdia,bell dsl internet - atlantic,1022,176.0,175520.0,997.272727
45895,net-67-225-28-0-1,wanec-1005,67.225.28.0/24,67.225.28.0 - 67.225.28.255,reassigned,NaN,swanec,swanec,sasktel wide area network engineering center,254,975.0,320780.0,329.005128
54165,net-65-93-30-0-1,dsl-65-93-30-0-1603821250,65.93.30.0/24,65.93.30.0 - 65.93.30.255,reassigned,as577,bdio,bdio,bell dsl internet ontario,254,4286.0,1088644.0,254.000000


In [23]:
sorted_filtered.drop_duplicates(subset=['org_name'])[['org_name', 'cidr_count' ,'avg_ip_per_cidr']]

,org_name,cidr_count,avg_ip_per_cidr
47515,bell canada,116.0,76120.482759
35138,rogers communications canada inc.,125.0,46936.112000
6955,telus communications inc.,107.0,46539.757009
13236,shared services canada,336.0,13422.761905
8389,"ovh hosting, inc.",792.0,1232.767677
36452,distributel communications ltd.,371.0,1199.832884
57787,"bell mobility, inc.",219.0,1170.821918
479,bell dsl internet - atlantic,176.0,997.272727
45895,sasktel wide area network engineering center,975.0,329.005128
54165,bell dsl internet ontario,4286.0,254.000000


In [32]:
sorted_filtered.groupby('org_name')['avg_ip_per_cidr'].sum()

org_name
bell canada                                     76120.482759
bell dsl internet - atlantic                      997.272727
bell dsl internet ontario                         254.000000
bell dsl internet quebec                          254.000000
bell mobility, inc.                              1170.821918
distributel communications ltd.                  1199.832884
ovh (nwk)                                         217.879518
ovh hosting, inc.                                1232.767677
rogers communications canada inc.               46936.112000
sasktel wide area network engineering center      329.005128
shared services canada                          13422.761905
telus communications inc.                       46539.757009
Name: avg_ip_per_cidr, dtype: float64

In [34]:
sorted_filtered.sort_values(by='avg_ip_per_cidr', ascending=False).head(10)

,net_id,net_name,cidr,net_range,net_type,asn,org_cust_id,org_id,org_name,ip_count,cidr_count,avg_ip_per_cidr
63378,net-99-224-0-0-1,rogers-com-hsd,99.224.0.0/11,99.224.0.0 - 99.255.255.255,direct allocation,as812,rcc-182,rcc-182,rogers communications canada inc.,2097150,125.0,16777.200000
8653,net-142-112-0-0-1,bell-icnen21,142.112.0.0/12,142.112.0.0 - 142.127.255.255,direct allocation,as577,linx,linx,bell canada,1048574,116.0,9039.431034
33750,net-99-208-0-0-1,rogers-com-dps,99.208.0.0/12,99.208.0.0 - 99.223.255.255,direct allocation,as812,rcc-182,rcc-182,rogers communications canada inc.,1048574,125.0,8388.592000
61770,net-75-152-0-0-1,telus,75.152.0.0/13,75.152.0.0 - 75.159.255.255,direct allocation,NaN,tace,tace,telus communications inc.,524286,107.0,4899.869159
22817,net-184-144-0-0-1,bellcanada-20,184.144.0.0/13,184.144.0.0 - 184.151.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,4519.706897
29378,net-174-88-0-0-1,bellcanada-19,174.88.0.0/13,174.88.0.0 - 174.95.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,4519.706897
47515,net-70-24-0-0-1,bellcanada-18,70.24.0.0/13,70.24.0.0 - 70.31.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,4519.706897
50239,net-70-48-0-0-1,bellnexxia-11,70.48.0.0/13,70.48.0.0 - 70.55.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,4519.706897
46835,net-76-64-0-0-1,bellcanada-16,76.64.0.0/13,76.64.0.0 - 76.71.255.255,direct allocation,NaN,linx,linx,bell canada,524286,116.0,4519.706897
11661,net-174-112-0-0-1,rogers-com-hsd,174.112.0.0/13,174.112.0.0 - 174.119.255.255,direct allocation,as812,rcc-182,rcc-182,rogers communications canada inc.,524286,125.0,4194.288000
